# 3. Train-Predict

**Tensorboard**
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

In [3]:
import time
import os
import pandas as pd

project_name = 'Dog_Breed_Identification'
step_name = 'Train-Predict'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

cwd = os.getcwd()
log_path = os.path.join(cwd, 'log')
model_path = os.path.join(cwd, 'model')
output_path = os.path.join(cwd, 'output')
print('log_path: \t' + log_path)
print('model_path: \t' + model_path)
print('output_path: \t' + output_path)

run_name: Dog_Breed_Identification_Train-Predict_20171102_221706
log_path: 	D:\Udacity\MachineLearning(Advanced)\p6_graduation_project\log
model_path: 	D:\Udacity\MachineLearning(Advanced)\p6_graduation_project\model
output_path: 	D:\Udacity\MachineLearning(Advanced)\p6_graduation_project\output


In [17]:
aa = 'qwasdf'
print(list(aa))

['q', 'w', 'a', 's', 'd', 'f']


In [16]:
import numpy as np
a = np.zeros((10357,2048))
b = np.zeros((10357,2048))
c = np.dot(a,b.transpose())
print(c.shape)
d = np.zeros((c.shape[0]*c.shape[1]))
print(d.shape)

(10357, 10357)
(107267449,)


In [2]:
df = pd.read_csv(os.path.join(cwd, 'input', 'labels.csv'))
print('lables amount: %d' %len(df))
df.head()

FileNotFoundError: File b'D:\\Udacity\\MachineLearning(Advanced)\\p6_graduation_project\\input\\labels.csv' does not exist

In [ ]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

x_train = []
y_train = {}
x_val = []
y_val = {}
x_test = []

cwd = os.getcwd()
feature_cgg16 = os.path.join(cwd, 'model', 'feature_VGG16_{}.h5'.format(20171026))
feature_cgg19 = os.path.join(cwd, 'model', 'feature_VGG19_{}.h5'.format(20171026))
feature_resnet50 = os.path.join(cwd, 'model', 'feature_ResNet50_{}.h5'.format(20171026))
feature_xception = os.path.join(cwd, 'model', 'feature_Xception_{}.h5'.format(20171026))
feature_inception = os.path.join(cwd, 'model', 'feature_InceptionV3_{}.h5'.format(20171026))
# feature_inceptionResNetV2 = os.path.join(cwd, 'model', 'feature_InceptionResNetV2_{}.h5'.format(20171028))
for filename in [feature_cgg16, feature_cgg19, feature_resnet50, feature_xception, feature_inception]:
    with h5py.File(filename, 'r') as h:
        x_train.append(np.array(h['train']))
        y_train = np.array(h['train_labels'])
        x_test.append(np.array(h['test']))

# print(x_train[0].shape)
x_train = np.concatenate(x_train, axis=-1)
# y_train = np.concatenate(y_train, axis=0)
# x_val = np.concatenate(x_val, axis=-1)
# y_val = np.concatenate(y_val, axis=0)
x_test = np.concatenate(x_test, axis=-1)
print(x_train.shape)
print(x_train.shape[1:])

print(len(y_train))
# print(x_val.shape)
# print(len(y_val))
print(x_test.shape)

In [ ]:
from sklearn.utils import shuffle
(x_train, y_train) = shuffle(x_train, y_train)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.05, random_state=2017)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [6]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(y_train.shape)
print(y_val.shape)

Using TensorFlow backend.


(9710, 120)
(512, 120)


### Build CNN

In [7]:
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [8]:
log_dir = os.path.join(log_path, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:E:\Udacity\MachineLearning(Advanced)\p6_graduation_project\log\Dog_Breed_Identification_Train-Predict_20171028_225523


In [9]:
model = Sequential()
model.add(Dense(8192, input_shape=x_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(8192, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))

model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
hist = model.fit(x_train, y_train,
                 batch_size=32,
                 epochs=20, #Increase this when not on Kaggle kernel
                 verbose=1,  #1 for ETA, 0 for silent
                 validation_data=(x_val, y_val), 
                 callbacks=[tensorBoard])

Train on 9710 samples, validate on 512 samples
Epoch 1/20
9710/9710 [==============================] - 83s - loss: 3.0643 - acc: 0.2966 - val_loss: 1.5338 - val_acc: 0.5371
Epoch 2/20
9710/9710 [==============================] - 83s - loss: 1.2247 - acc: 0.6448 - val_loss: 1.1914 - val_acc: 0.6445
Epoch 3/20
9710/9710 [==============================] - 83s - loss: 0.7921 - acc: 0.7639 - val_loss: 0.9403 - val_acc: 0.6895
Epoch 4/20
9710/9710 [==============================] - 83s - loss: 0.5387 - acc: 0.8344 - val_loss: 0.8295 - val_acc: 0.7539
Epoch 5/20
9710/9710 [==============================] - 83s - loss: 0.4294 - acc: 0.8636 - val_loss: 0.7411 - val_acc: 0.7871
Epoch 6/20
9710/9710 [==============================] - 83s - loss: 0.3293 - acc: 0.8940 - val_loss: 0.7907 - val_acc: 0.7988
Epoch 7/20
9710/9710 [==============================] - 83s - loss: 0.2892 - acc: 0.9074 - val_loss: 0.8161 - val_acc: 0.7480
Epoch 8/20
9710/9710 [==============================] - 83s - loss: 0.2

KeyboardInterrupt: 

In [ ]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
run_name0 = run_name + '_' + str(int(final_acc*10000)).zfill(4)

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name0)

## Predict

In [ ]:
# Used to load model directly and skip train
# import os
# from keras.models import load_model
# cwd = os.getcwd()
# model = load_model(os.path.join(cwd, 'model', 'Dog_Breed_Identification_Train_20171024_155154.h5'))

In [ ]:
y_pred = model.predict(x_test, batch_size=128)
print(y_pred.shape)

In [ ]:
# print(y_pred[:10])
# y_pred = np.clip(y_pred, 0.005, 0.995)
# print(y_pred[:10])

In [ ]:
files = os.listdir(os.path.join(cwd, 'input', 'data_test', 'test'))
print(files[:10])

In [ ]:
cwd = os.getcwd()
df = pd.read_csv(os.path.join(cwd, 'input', 'labels.csv'))
print('lables amount: %d' %len(df))
df.head()

In [ ]:
n = len(df)
breed = set(df['breed'])
n_class = len(breed)
class_to_num = dict(zip(breed, range(n_class)))
num_to_class = dict(zip(range(n_class), breed))
print(breed)

In [ ]:
df2 = pd.read_csv('.\\input\\sample_submission.csv')
n_test = len(df2)
print(df2.shape)

In [ ]:
for i in range(0, 120):
    df2.iloc[:,[i+1]] = y_pred[:,i]
if not os.path.exists(output_path):
    os.mkdir(output_path)
pred_file = os.path.join(output_path, 'pred_' + run_name0 + '.csv')
df2.to_csv(pred_file, index=None)

In [ ]:
print(run_name0)
print('Done !')